<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/Minesweeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minesweeper Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

The player is presented with a grid of squares. Some randomly selected squares, unknown to the player, contain mines. Other squares may display numbers indicating the count of mines present in the immediate neighborhood of the squares. Find all mines.

See https://de.wikipedia.org/wiki/Minesweeper or go to http://minesweeperonline.com/ to play

In [3]:
!pip install ortools

In [4]:
from ortools.sat.python import cp_model
from itertools import product

Some Minesweeper examples, the first one is from the lecture

Pick one of the examples

Create model

In [66]:
mine_field1 = [
    ["2", "", "", "", "1"],
    ["2", "", "4", "3", ""],
    ["", "2", "", "1", ""],
    ["", "1", "", "3", ""],
    ["1", "", "", "", ""],
]
mine_field2 = [
    ["1", "", "", "", "2", "", "", "1"],
    ["", "1", "2", "", "3", "", "", "1"],
    ["", "2", "", "1", "", "", "", "0"],
    ["", "2", "1", "", "", "2", "3", "1"],
    ["", "", "", "2", "", "", "", ""],
    ["1", "", "", "", "4", "3", "", ""],
    ["", "1", "", "", "4", "", "3", ""],
    ["1", "", "", "2", "", "2", "", "1"],
]
mine_field3 = [
    ["", "", "", "", "", "1", "", ""],
    ["1", "2", "2", "1", "", "", "1", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "0", "", "", "1", "", "1"],
    ["", "1", "", "", "", "", "", ""],
    ["", "", "1", "", "2", "3", "", ""],
    ["", "", "", "", "", "", "2", "1"],
    ["", "1", "", "2", "2", "", "2", ""],
]
mine_field4 = [
    ["1", "", "", "2", "", "2", "", "2", "", ""],
    ["", "3", "2", "", "", "", "4", "", "", "1"],
    ["", "", "", "1", "3", "", "", "", "4", ""],
    ["3", "", "1", "", "", "", "3", "", "", ""],
    ["", "2", "1", "", "1", "", "", "3", "", "2"],
    ["", "3", "", "2", "", "", "2", "", "1", ""],
    ["2", "", "", "3", "2", "", "", "2", "", ""],
    ["", "3", "", "", "", "3", "2", "", "", "3"],
    ["", "", "3", "", "3", "3", "", "", "", ""],
    ["", "2", "", "2", "", "", "", "", "2", ""],
]

game = mine_field3
model = cp_model.CpModel()

game_row_count = len(game)
game_col_count = len(game[0])

# Type your model here ...
# treat mines seperately makes the assignment later easier
mines = [[model.NewBoolVar(f"is_a_mine({i},{j})") for j in range(game_col_count)] for i in range(game_row_count)]
board = [[model.NewIntVar(0, 8, f"({i},{j})") for j in range(game_col_count)] for i in range(game_row_count)]

# preassign numbers
for i, row in enumerate(game):
    for j, cell in enumerate(row):
        if cell != '':
          model.Add(board[i][j] == int(cell))
          model.Add(mines[i][j] == False)

# check neighbourhoods
for i in range(game_row_count):
  for j in range(game_col_count):
    neighbor_mines = []
    for di in [-1, 0, 1]:  # those are the indizes around the field
      for dj in [-1, 0, 1]: # those as wellare the indizes around the field
        ## not itself AND needs to be on board
        ni = i + di
        nj = j + dj
        if (di != 0 or dj != 0) and (0 <= ni < game_row_count) and (0 <= nj < game_col_count):
          neighbor_mines.append(mines[ni][nj])
    constraint = model.Add(sum(neighbor_mines) == board[i][j])
    # todo: enable and check impact
    # constraint.OnlyEnforceIf(mines[i][j].Not()) ## the issue here is, that we suddenly get duplicated solutions





Callback for solution printing

In [11]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables, game):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables

    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                if self.Value(self.__variables[i][j]) == 1:
                    print("[💣]\t", end='')
                elif game[i][j] == "":
                    print("[  ]\t", end='')
                else:
                    print("[ {}]\t".format(game[i][j]), end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [67]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(mines, game))
print(status)

[  ]	[💣]	[💣]	[  ]	[  ]	[ 1]	[💣]	[  ]	

[ 1]	[ 2]	[ 2]	[ 1]	[  ]	[  ]	[ 1]	[ 1]	

[  ]	[ 1]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	

[💣]	[  ]	[ 0]	[  ]	[  ]	[ 1]	[  ]	[ 1]	

[  ]	[ 1]	[  ]	[  ]	[  ]	[  ]	[💣]	[  ]	

[  ]	[  ]	[ 1]	[  ]	[ 2]	[ 3]	[  ]	[  ]	

[  ]	[  ]	[💣]	[  ]	[💣]	[💣]	[ 2]	[ 1]	

[  ]	[ 1]	[  ]	[ 2]	[ 2]	[  ]	[ 2]	[💣]	




4


In [54]:
print(f"Runtime:   {solver.WallTime()}ms")
print(f"Booleans:  {solver.NumBooleans()}")
print(f"Failures:  {solver.NumConflicts()}")
print(f"Branches:  {solver.NumBranches()}")


Runtime:   0.005506531ms
Booleans:  0
Failures:  0
Branches:  0


In [55]:
# game 1
# with still to many results, nr 1
# Runtime:   0.6281455770000001ms
# Booleans:  22
# Failures:  30
# Branches:  968

# with the right results
# Runtime:   0.00293362ms
# Booleans:  0
# Failures:  0
# Branches:  0
## bool var
# Runtime:   0.011351261000000001ms
# Booleans:  0
# Failures:  0
# Branches:  0

# game 2
# Runtime:   0.007256221ms
# Booleans:  0
# Failures:  0
# Branches:  0
## boolVar
# Runtime:   0.0078048710000000006ms
# Booleans:  0
# Failures:  0
# Branches:  0

# game 3
# Runtime:   0.020923257ms
# Booleans:  0
# Failures:  0
# Branches:  0
## boolVar
# Runtime:   0.013626804000000001ms
# Booleans:  0
# Failures:  0
# Branches:  0

# game 4
# Runtime:   0.031164788000000002ms
# Booleans:  0
# Failures:  0
# Branches:  0
## bool var
# Runtime:   0.012166152000000001ms
# Booleans:  0
# Failures:  0
# Branches:  0